In [ ]:

import math
import pandas as pd
from pulp import (
    LpProblem, LpMinimize, LpVariable, lpSum, LpBinary,
    LpStatus, value, PULP_CBC_CMD
)

# =========================
# 1) INPUT DATA (Section 3)
# =========================

# Plants I: lat, lon, capacity (tons/year)
plants = {
    "I1_ElJadida": {"lat": 33.2316, "lon": -8.5007, "cap": 600_000},
    "I2_Meknes":   {"lat": 33.8730, "lon": -5.5407, "cap": 500_000},
}


# Warehouses J: lat, lon, capacity (tons/year), fixed opening cost (MAD/year)
# Feasibility fix: capacities increased to avoid infeasibility (demand > capacity)
warehouses = {
    "J1_Casablanca": {"lat": 33.5731, "lon": -7.5898, "cap": 250_000, "fixed": 9_000_000},
    "J2_Rabat":      {"lat": 34.0209, "lon": -6.8416, "cap": 170_000, "fixed": 6_500_000},
    "J3_Fes":        {"lat": 34.0181, "lon": -5.0078, "cap": 155_000, "fixed": 5_800_000},
    "J4_Marrakech":  {"lat": 31.6295, "lon": -7.9811, "cap": 180_000, "fixed": 6_200_000},
    "J5_Agadir":     {"lat": 30.4278, "lon": -9.5981, "cap": 130_000, "fixed": 5_000_000},
}

# Demand zones K (regional centroids): lat, lon
markets = {
    "K1_CasablancaSettat": {"lat": 33.5731, "lon": -7.5898},
    "K2_RabatSaleKenitra": {"lat": 34.0209, "lon": -6.8416},
    "K3_TangierTetouan":   {"lat": 35.7595, "lon": -5.8340},
    "K4_FesMeknes":        {"lat": 34.0181, "lon": -5.0078},
    "K5_MarrakechSafi":    {"lat": 31.6295, "lon": -7.9811},
    "K6_SoussMassa":       {"lat": 30.4278, "lon": -9.5981},
}

# Products P
products = ["P1_Milk", "P2_Yogurt"]

# Base annual demand (tons/year) by (market, product)
base_demand = {
    ("K1_CasablancaSettat", "P1_Milk"): 180_000,
    ("K1_CasablancaSettat", "P2_Yogurt"): 140_000,
    ("K2_RabatSaleKenitra", "P1_Milk"): 95_000,
    ("K2_RabatSaleKenitra", "P2_Yogurt"): 70_000,
    ("K3_TangierTetouan", "P1_Milk"): 80_000,
    ("K3_TangierTetouan", "P2_Yogurt"): 60_000,
    ("K4_FesMeknes", "P1_Milk"): 70_000,
    ("K4_FesMeknes", "P2_Yogurt"): 55_000,
    ("K5_MarrakechSafi", "P1_Milk"): 85_000,
    ("K5_MarrakechSafi", "P2_Yogurt"): 65_000,
    ("K6_SoussMassa", "P1_Milk"): 60_000,
    ("K6_SoussMassa", "P2_Yogurt"): 45_000,
}

# Cost parameters
cost_per_ton_km_PW = 1.8  # Plant → Warehouse (MAD/ton/km)
cost_per_ton_km_WM = 2.3  # Warehouse → Market (MAD/ton/km)
holding_cost = {"P1_Milk": 1200, "P2_Yogurt": 1600}  # MAD/ton/year

# =========================
# 2) SCENARIOS (Section 4)
# =========================

demand_scenarios = {
    "S1": 1.00,  # base
    "S2": 1.15,  # high demand
    "S3": 0.85,  # low demand
}

cost_scenarios = {
    "C1": 1.00,  # base transport costs
    "C2": 1.20,  # +20% transport costs
    "C3": 0.85,  # -15% transport costs
}

combined_scenarios = {
    "CS1_S1C1": ("S1", "C1"),
    "CS2_S2C1": ("S2", "C1"),
    "CS3_S3C1": ("S3", "C1"),
    "CS4_S1C2": ("S1", "C2"),
    "CS5_S1C3": ("S1", "C3"),
}

# =========================
# 3) DISTANCES (Haversine)
# =========================

def haversine_km(lat1, lon1, lat2, lon2):
    R = 6371.0
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dl = math.radians(lon2 - lon1)
    a = math.sin(dphi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(dl / 2) ** 2
    return 2 * R * math.asin(math.sqrt(a))

dist_PW = {(i, j): haversine_km(plants[i]["lat"], plants[i]["lon"],
                               warehouses[j]["lat"], warehouses[j]["lon"])
           for i in plants for j in warehouses}

dist_WM = {(j, k): haversine_km(warehouses[j]["lat"], warehouses[j]["lon"],
                               markets[k]["lat"], markets[k]["lon"])
           for j in warehouses for k in markets}

def get_scenario_demand(cs_name):
    sD, _ = combined_scenarios[cs_name]
    mult = demand_scenarios[sD]
    return {(k, p): base_demand[(k, p)] * mult for k in markets for p in products}

def get_transport_multiplier(cs_name):
    _, sC = combined_scenarios[cs_name]
    return cost_scenarios[sC]

# =========================
# 4) SOLVER FUNCTION
# =========================

def solve_scenario(cs_name, solver=None):
    dem = get_scenario_demand(cs_name)
    t_mult = get_transport_multiplier(cs_name)

    model = LpProblem(f"NetworkDesign_{cs_name}", LpMinimize)

    # Variables
    y = LpVariable.dicts("OpenWH", list(warehouses.keys()), lowBound=0, upBound=1, cat=LpBinary)
    x = LpVariable.dicts("FlowPW",
                         [(i, j, p) for i in plants for j in warehouses for p in products],
                         lowBound=0)
    z = LpVariable.dicts("FlowWM",
                         [(j, k, p) for j in warehouses for k in markets for p in products],
                         lowBound=0)

    # Objective
    fixed = lpSum(warehouses[j]["fixed"] * y[j] for j in warehouses)

    pw = lpSum((cost_per_ton_km_PW * t_mult * dist_PW[(i, j)]) * x[(i, j, p)]
               for i in plants for j in warehouses for p in products)

    wm = lpSum((cost_per_ton_km_WM * t_mult * dist_WM[(j, k)]) * z[(j, k, p)]
               for j in warehouses for k in markets for p in products)

    hold = lpSum(holding_cost[p] * lpSum(z[(j, k, p)] for k in markets)
                 for j in warehouses for p in products)

    model += fixed + pw + wm + hold

    # Constraints

    # Demand satisfaction
    for k in markets:
        for p in products:
            model += lpSum(z[(j, k, p)] for j in warehouses) == dem[(k, p)], f"Demand_{k}_{p}"

    # Warehouse flow balance per product
    for j in warehouses:
        for p in products:
            model += lpSum(x[(i, j, p)] for i in plants) == lpSum(z[(j, k, p)] for k in markets), f"FlowBal_{j}_{p}"

    # Plant capacity
    for i in plants:
        model += lpSum(x[(i, j, p)] for j in warehouses for p in products) <= plants[i]["cap"], f"PlantCap_{i}"

    # Warehouse capacity (throughput) conditional on opening
    for j in warehouses:
        model += lpSum(z[(j, k, p)] for k in markets for p in products) <= warehouses[j]["cap"] * y[j], f"WHCap_{j}"

    # Solve
    if solver is None:
        solver = PULP_CBC_CMD(msg=False)
    model.solve(solver)

    status = LpStatus[model.status]

    # Extract decisions (only meaningful if feasible/optimal)
    opened = {j: int(round(value(y[j]))) if status in ("Optimal", "Feasible") else None for j in warehouses}

    # Safe cost extraction (avoid misleading costs on infeasible)
    def safe_val(expr):
        return float(value(expr)) if status in ("Optimal", "Feasible") else None

    fixed_cost = safe_val(lpSum(warehouses[j]["fixed"] * y[j] for j in warehouses))
    pw_cost = safe_val(lpSum((cost_per_ton_km_PW * t_mult * dist_PW[(i, j)]) * x[(i, j, p)]
                             for i in plants for j in warehouses for p in products))
    wm_cost = safe_val(lpSum((cost_per_ton_km_WM * t_mult * dist_WM[(j, k)]) * z[(j, k, p)]
                             for j in warehouses for k in markets for p in products))
    holding = safe_val(lpSum(holding_cost[p] * lpSum(z[(j, k, p)] for k in markets)
                             for j in warehouses for p in products))
    total = safe_val(model.objective)

    # Tables
    df_open = pd.DataFrame([{"scenario": cs_name, "warehouse": j, "open": opened[j]} for j in warehouses])

    df_pw = pd.DataFrame([{
        "scenario": cs_name, "plant": i, "warehouse": j, "product": p,
        "flow_tons": float(value(x[(i, j, p)])) if status in ("Optimal", "Feasible") else None
    } for i in plants for j in warehouses for p in products])

    df_wm = pd.DataFrame([{
        "scenario": cs_name, "warehouse": j, "market": k, "product": p,
        "flow_tons": float(value(z[(j, k, p)])) if status in ("Optimal", "Feasible") else None
    } for j in warehouses for k in markets for p in products])

    summary = {
        "scenario": cs_name,
        "status": status,
        "total_cost": total,
        "fixed_cost": fixed_cost,
        "pw_cost": pw_cost,
        "wm_cost": wm_cost,
        "holding_cost": holding,
    }

    return summary, df_open, df_pw, df_wm

# =========================
# 5) RUN ALL SCENARIOS + SAVE
# =========================

summaries = []
open_tables = []
pw_tables = []
wm_tables = []

for cs_name in combined_scenarios:
    summary, df_open, df_pw, df_wm = solve_scenario(cs_name)
    summaries.append(summary)
    open_tables.append(df_open)
    pw_tables.append(df_pw)
    wm_tables.append(df_wm)

df_cost = pd.DataFrame(summaries).sort_values("scenario")
df_open = pd.concat(open_tables, ignore_index=True)
df_pw = pd.concat(pw_tables, ignore_index=True)
df_wm = pd.concat(wm_tables, ignore_index=True)

df_cost.to_csv("Table6_cost_summary.csv", index=False)
df_open.to_csv("Table6_opening_decisions.csv", index=False)
df_pw.to_csv("Appendix_flows_PW.csv", index=False)
df_wm.to_csv("Appendix_flows_WM.csv", index=False)

print(df_cost)

# Optional: quick robustness view (how often each warehouse opens)
if df_open["open"].notna().any():
    print("\nWarehouse opening frequency across solved scenarios:")
    print(df_open.pivot_table(index="warehouse", values="open", aggfunc="mean").sort_values("open", ascending=False))
